In [1]:
import pandas as pd
import numpy as np
import time
import sys

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression  
from sklearn import svm

module_path='/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Experiment(Supervised Learning)/evaluation/'
if module_path not in sys.path:
    sys.path.append(module_path)
    
from integrating_embedding import *

import integrating_embedding
from imp import reload
reload(integrating_embedding)

from analysis_cohort import *

import analysis_cohort
from imp import reload
reload(analysis_cohort)

emb_path_trn_mimic = '/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Cohort_selection/Train/result/emb_train_new_version.npy'
emb_path_vld_mimic = '/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Cohort_selection/Train/result/emb_valid_new_version.npy'
emb_path_event_mimic = '/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Cohort_selection/Train/result/emb_event_new_version.npy'

emb_path_trn_eicu = '/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Cohort_selection/Train/result/emb_eicu_new_version.npy'

mimic_path = '/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Case Labeling/mimic_analysis.csv.gz'
eicu_path = '/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Case Labeling/eicu_analysis.csv.gz'

mimic_train_emb, mimic_valid_emb, event, event_emb = integrating_embedding.integrating(mimic_path, emb_path_trn_mimic, emb_path_vld_mimic, emb_path_event_mimic, 'mimic')
eicu_test_emb, event_eicu = integrating_embedding.integrating(eicu_path, emb_path_trn_eicu, _, _, 'eicu')

c:\Users\DAHS\anaconda3\envs\DL\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


========== 데이터셋 분할 정보 ==========
데이터셋 비율: 학습 = 0.90, 테스트 = 0.10
학습셋 클래스 비율: Case
1.0    473941
2.0      5128
3.0     11145
4.0     19613
Name: count, dtype: int64
테스트셋 클래스 비율: Case
1.0    53103
2.0      512
3.0      873
4.0     1590
Name: count, dtype: int64
--------------------------------------
========== 클래스 비율 ==========
학습셋 클래스 비율: 0.93:0.01:0.02:0.04
테스트셋 클래스 비율: 0.95:0.01:0.02:0.03
--------------------------------------
========== 환자 및 체류 정보 ==========
학습셋 환자 수: 2907
테스트셋 환자 수: 329
학습셋 체류 수: 2961
테스트셋 체류 수: 330
--------------------------------------
========== 실험 설정 ==========
분할 시드: 9040
학습 비율: 0.9
임계값: 0.05
--------------------------------------
========== 실행 결과 ==========
총 소요 시간(초): 0.39
시도한 시행 횟수: 0


In [108]:
module_path='/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Experiment(Supervised Learning)/evaluation/'
if module_path not in sys.path:
    sys.path.append(module_path)

import split
from imp import reload
reload(split)

import Multiclass_evaluation
reload(Multiclass_evaluation)

<module 'Multiclass_evaluation' from 'c:\\Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Experiment(Supervised Learning)/evaluation\\Multiclass_evaluation.py'>

In [104]:
X_train, y_train, _ = split.split_X_Y(mimic_train_emb, mode = 'mimic')
X_valid, y_valid, valid_output = split.split_X_Y(mimic_valid_emb, mode = 'mimic')
X_test, y_test, test_output = split.split_X_Y(eicu_test_emb, mode = 'eicu')

In [109]:
Multiclass_evaluation.create_analysis(event, event_eicu, X_train, y_train, X_valid, X_test, valid_output, test_output)

|MIMIC-Validation|


,Model,Accuracy,Case 1 Score,Case 2 Score,Case 3 Score,Case 4 Score
0,xgb,0.984575,0.9955,0.8699,0.8504,0.9180
1,lgbm,0.980117,0.9943,0.7682,0.7844,0.8514
2,rf,0.985057,0.9963,0.8291,0.8457,0.9150
3,dt,0.974072,0.9885,0.7430,0.7167,0.8519


----------------------------------------------------------------------
|eICU-test|


,Model,Accuracy,Case 1 Score,Case 2 Score,Case 3 Score,Case 4 Score
0,xgb,0.989507,0.9971,0.3182,0.8686,0.9644
1,lgbm,0.973362,0.9921,0.3916,0.4863,0.6024
2,rf,0.991242,0.9989,0.1952,0.8989,0.9716
3,dt,0.895800,0.9036,0.2842,0.5776,0.8656


In [ ]:
#class 갯수가 많은 애들은 contrastive 쓰나 마나 똑같음 -> 원본데이터로도 충분히 정보가 있음
#하지만 contrastive 를 씀으로써 맞추기 힘든 case 2를 잘 맞출 수 있게 됨 -> 모든 모델에서 모두 상승함
# contastive learning이 데이터 불균형 상황이더라도, 소수 클래스에 대해서 적합을 잘할 수 있음

# 이를 바탕으로 쇼크 모니터링 시스템을 개발함
# 베스 이스라엘 뿐 아니라, 다른 다양한 병원에서도 잘 적합하는 시스템을 개발하기 위해 본 연구에서는 eicu 데이터를 활용
# 따라서 eicu 2014를 이용해 시스템을 개발하고, 성능 평가를 위해 2015 데이터를 활용
# 가장 성능이 좋았던 xgbm 을 기반으로 시스템을 개발